In [8]:
import pandas as pd
import os
import pyproj
from pyproj import Transformer

In [9]:
help(pyproj.transform)

Help on function transform in module pyproj.transformer:

transform(p1: Any, p2: Any, x: Any, y: Any, z: Optional[Any] = None, tt: Optional[Any] = None, radians: bool = False, errcheck: bool = False, always_xy: bool = False)
    .. versionadded:: 2.2.0 always_xy
    
    .. deprecated:: 2.6.1
        This function is deprecated. See: :ref:`upgrade_transformer`
    
    x2, y2, z2 = transform(p1, p2, x1, y1, z1)
    
    Transform points between two coordinate systems defined by the
    Proj instances p1 and p2.
    
    The points x1,y1,z1 in the coordinate system defined by p1 are
    transformed to x2,y2,z2 in the coordinate system defined by p2.
    
    z1 is optional, if it is not set it is assumed to be zero (and
    only x2 and y2 are returned). If the optional keyword
    'radians' is True (default is False), then all input and
    output coordinates will be in radians instead of the default
    of degrees for geographic input/output projections.
    If the optional keyword 'er

In [21]:
df_marks = pd.read_csv('../DATA/GCP_IMAGES/101_0001/101_0001_GCP.MRK')

# Parse out the filename from the path
df_marks['filename']=[os.path.basename(p).split('\\')[1] for p in df_marks['Path']]
df_marks = df_marks.set_index('filename')

#tidy up ellipoidal height
df_marks['EllipsoideHight'] = df_marks['EllipsoideHight'].str.replace(',Ellh','').astype('float')

# Convert the marks to Eastings and Northings
transformer = Transformer.from_crs(pyproj.CRS('EPSG:4327').to_3d(), 
                                   pyproj.CRS('EPSG:7850').to_3d(), 
                                   always_xy=True)
df_marks['Eastings'], df_marks['Northings'], df_marks['Z'] = transformer.transform(df_marks['Longitude'],df_marks['Latitude'],df_marks['EllipsoideHight'])

drone_location = df_marks[['Eastings','Northings','Z']]
drone_location

,Eastings,Northings,Z
filename,,,
101_0001_0002.JPG,394544.202734,6.459567e+06,56.224
101_0001_0021.JPG,394631.826848,6.459659e+06,56.058
101_0001_0068.JPG,394511.807591,6.459683e+06,56.266
101_0001_0099.JPG,394566.343655,6.459685e+06,56.211
101_0001_0113.JPG,394645.847291,6.459615e+06,56.227


In [28]:
df_meta = pd.read_csv('../DATA/GCP_IMAGES/101_0001/drone.csv')
#Parse out the filename from the path
df_meta['filename']=[os.path.basename(p) for p in df_meta['SourceFile']]
df_meta = df_meta.set_index('filename')
gimbal_angles = df_meta[['XMP:GimbalYawDegree', 'XMP:GimbalPitchDegree', 'XMP:FlightRollDegree']]
gimbal_angles = gimbal_angles.rename(columns={'XMP:GimbalYawDegree':'Yaw',
                                      'XMP:GimbalPitchDegree':'Pitch',
                                      'XMP:FlightRollDegree':'Roll'})
gimbal_angles

,Yaw,Pitch,Roll
filename,,,
101_0001_0002.JPG,55.1,-60.0,0.8
101_0001_0021.JPG,-124.9,-60.0,4.9
101_0001_0068.JPG,145.2,-60.0,14.0
101_0001_0099.JPG,145.0,-60.0,11.3
101_0001_0113.JPG,-38.1,-60.0,-13.5


In [29]:
# Join the dataframes on filename
df_combined = drone_location.join(gimbal_angles)
df_combined

,Eastings,Northings,Z,Yaw,Pitch,Roll
filename,,,,,,
101_0001_0002.JPG,394544.202734,6.459567e+06,56.224,55.1,-60.0,0.8
101_0001_0021.JPG,394631.826848,6.459659e+06,56.058,-124.9,-60.0,4.9
101_0001_0068.JPG,394511.807591,6.459683e+06,56.266,145.2,-60.0,14.0
101_0001_0099.JPG,394566.343655,6.459685e+06,56.211,145.0,-60.0,11.3
101_0001_0113.JPG,394645.847291,6.459615e+06,56.227,-38.1,-60.0,-13.5


In [ ]:
# Setup the camera for each acquisition and project
